# COMP90051 Project 2

In [1]:
# Do not edit. These are the only imports permitted.
%matplotlib inline
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

## 1. Implementing ε-Greedy and UCB

In [2]:
class MAB(ABC):
    """
    Abstract class that represents a multi-armed bandit (MAB)
    """
    
    @abstractmethod
    def play(self, tround, context):
        """
        Play a round
        
        Arguments
        =========
        tround : int
            positive integer identifying the round
        
        context : 1D float array, shape (self.ndims * self.narms), optional
            context given to the arms
        
        Returns
        =======
        arm : int
            the positive integer arm id for this round
        """
    
    @abstractmethod
    def update(self, arm, reward, context):
        """
        Updates the internal state of the MAB after a play
        
        Arguments
        =========
        arm : int
            a positive integer arm id in {1, ..., self.narms}
        
        reward : float
            reward received from arm
        
        context : 1D float array, shape (self.ndims * self.narms), optional
            context given to arms
        """

In [3]:
class EpsGreedy(MAB):
    """
    Epsilon-Greedy multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    epsilon : float
        explore probability

    Q0 : float, optional
        initial value for the arms
    """
    
    def __init__(self, narms, epsilon, Q0=np.inf):
        
        self.narms = narms
        self.epsilon = epsilon
        self.Q0 = Q0
        self.arm_visit_count = {}
        self.arm_total_reward = {}
        
        self.arm_with_Q = {}
        for arm in range(1,self.narms+1):
            self.arm_with_Q[arm]=self.Q0 #Initial all the arm with Q0
            
            self.arm_visit_count[arm]=0 #Initial all the arm with number of visits
            self.arm_total_reward[arm] = 0
            
        
    def play(self, tround, context=None):
        if np.random.random() <self.epsilon:#exploration
            action = np.random.choice(self.narms)
        else:
            highest = max(self.arm_with_Q.values())
            highest_Qs = [key for key, value in self.arm_with_Q.items() if value == highest]
            if len(highest_Qs)>1:
                action = np.random.choice(highest_Qs)#Tie Breaker
            else:
                action = highest_Qs[0]
         
        return action
        
        
    def update(self, arm, reward, context=None):
        self.arm_visit_count[arm] += 1
        self.arm_total_reward[arm] += reward
        updated_reward = self.arm_total_reward[arm]/self.arm_visit_count[arm]
        
        self.arm_with_Q.update({arm:updated_reward})
        
        return self.arm_with_Q
        
        
    

In [4]:
class UCB(MAB):
    """
    Upper Confidence Bound (UCB) multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    rho : float
        positive real explore-exploit parameter

    Q0 : float, optional
        initial value for the arms
    """
    
    def __init__(self, narms, rho, Q0=np.inf):
        self.narms = narms
        self.rho = rho
        self.Q0 = Q0
        self.arm_visit_count = {}
        self.arm_total_reward = {}
        
        self.arm_with_avg_reward = {}
        for arm in range(1,self.narms+1):
            self.arm_with_avg_reward[arm]=self.Q0 #Initial all the arm with Q0
            
            self.arm_visit_count[arm]=0 #Initial all the arm with number of visits
            self.arm_total_reward[arm] = 0
        
    
    def play(self, tround, context=None):
        temp_arm_with_Q = self.arm_with_avg_reward
        
        
        
        for arm in temp_arm_with_Q:
            if self.arm_visit_count[arm] == 0:
                continue
                
                
            else:
                explore_boost_const = self.rho*np.log(tround)/self.arm_visit_count[arm]
                
                temp_arm_with_Q[arm] = temp_arm_with_Q[arm] + np.sqrt(explore_boost_const)
            
        #print(temp_arm_with_Q)
        highest = max(temp_arm_with_Q.values())
        highest_Qs = [key for key, value in temp_arm_with_Q.items() if value == highest]
        if len(highest_Qs)>1:
            action = np.random.choice(highest_Qs)#Tie Breaker
        else:
            action = highest_Qs[0]
        return action
        
    def update(self, arm, reward, context=None):
        self.arm_visit_count[arm] += 1
        self.arm_total_reward[arm] += reward
        updated_reward = self.arm_total_reward[arm]/self.arm_visit_count[arm]
        
        self.arm_with_avg_reward.update({arm:updated_reward})
        
        return self.arm_with_avg_reward
        
    

## 2. The Basic Thompson Bandit

In [5]:
class BetaThompson(MAB):
    """
    Beta-Bernoulli Thompson sampling multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    alpha0 : float, optional
        positive real prior hyperparameter

    beta0 : float, optional
        positive real prior hyperparameter
    """
    
    def __init__(self, narms, alpha0=1.0, beta0=1.0):
        self.narms = narms
        self.alpha0 = alpha0
        self.beta0 = beta0
        self.theta = np.random.beta(self.alpha0, self.beta0)
        
        
        self.arm_with_theta = {}
        self.arm_with_alpha0 = {}
        self.arm_with_beta0 = {}
        
        for arm in range(1,self.narms+1):
            self.arm_with_theta[arm]=self.theta #Initial all the arm with Q0
            self.arm_with_alpha0[arm]=self.alpha0
            self.arm_with_beta0[arm]=self.beta0
            
        
        
    
    def play(self, tround, context=None):
        for arm in range(1,self.narms+1):
            self.arm_with_theta[arm] = np.random.beta(self.arm_with_alpha0[arm]+1, self.arm_with_beta0[arm]+1)
        highest = max(self.arm_with_theta.values())
        highest_Qs = [key for key, value in self.arm_with_theta.items() if value == highest]
        if len(highest_Qs)>1:
            action = np.random.choice(highest_Qs)#Tie Breaker
        else:
            action = highest_Qs[0]
        return action
        
        
        
    def update(self, arm, reward, context=None):
        if reward ==1:
            self.arm_with_alpha0[arm] += 1
        else:
            self.arm_with_beta0[arm] += 1
        

## 3. Off-Policy Evaluation

In [6]:
arm_list = []
reward_list = []
features_list = []
num_of_events = 0
with open("dataset.txt","r") as f:
    dataset = f.readlines()
for line in dataset:
    num_of_events += 1
    #print(line)
    temp_line = line.split()
    #print(temp_line)
    arm = int(temp_line[0])
    reward = float(temp_line[1])
    features = temp_line[2:]
    features = list(map(float, features))
    #features = np.reshape(features,(10,10))
    
    arm_list.append(arm)
    reward_list.append(reward)
    features_list.append(features)
arms = np.array(arm_list)
rewards = np.array(reward_list)
contexts = np.array(features_list).reshape(num_of_events,(10*10))



    
def offlineEvaluate(mab, arms, rewards, contexts, nrounds=None):
    """
    Offline evaluation of a multi-armed bandit
    
    Arguments
    =========
    mab : instance of MAB
    
    arms : 1D int array, shape (nevents,) 
        integer arm id for each event
    
    rewards : 1D float array, shape (nevents,)
        reward received for each event
    
    contexts : 2D float array, shape (nevents, mab.narms*nfeatures)
        contexts presented to the arms (stacked horizontally) 
        for each event.
        
    nrounds : int, optional
        number of matching events to evaluate `mab` on.
    
    Returns
    =======
    out : 1D float array
        rewards for the matching events
    """
    h0 = []
    R0 = []
    """if nrounds is None or nrounds > num_of_events:
        events = num_of_events
    else:
        events = nrounds"""
    count = 0
    for event in range(num_of_events):
        if len(h0) == nrounds:
            
            break
        
        action = mab.play(len(h0)+1,contexts[event])
        if action == arms[event]:
            count +=1
            h0.append(event)
            R0.append(rewards[event])
            mab.update(arms[event],rewards[event],contexts[event])
        
    
    return R0
            

        
    

In [7]:
mab = EpsGreedy(10, 0.05)
results_EpsGreedy = offlineEvaluate(mab, arms, rewards, contexts, 800)
print(len(results_EpsGreedy))
print('EpsGreedy average reward', np.mean(results_EpsGreedy))

800
EpsGreedy average reward 0.19375


In [8]:
mab = UCB(10, 1.0)
results_UCB = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('UCB average reward', np.mean(results_UCB))

UCB average reward 0.10125


In [9]:
mab = BetaThompson(10, 1.0, 1.0)
results_BetaThompson = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('BetaThompson average reward', np.mean(results_BetaThompson))

BetaThompson average reward 0.22


## 4. Contextual Bandits - LinUCB

In [18]:
class LinUCB(MAB):
    """
    Contextual multi-armed bandit (LinUCB)

    Arguments
    =========
    narms : int
        number of arms

    ndims : int
        number of dimensions for each arm's context

    alpha : float
        positive real explore-exploit parameter
    """
    
    def __init__(self, narms, ndims, alpha):
        self.narms = narms
        self.ndims = ndims
        self.alpha = alpha

        
        self.A_a = {}
        self.b_a = {}
        
        
        for arm in range(1,self.narms+1):
            if arm not in self.A_a:
                self.A_a[arm] = np.identity(self.ndims)
            if arm not in self.b_a:
                self.b_a[arm] = np.zeros(self.ndims)
            
         
            
        
        
    def play(self, tround, context):
        arm_with_Q = {}
        context = context.reshape(self.narms,self.ndims)
        
        for arm in range(1,self.narms+1):
            
            Theta_a = np.dot(np.linalg.inv(self.A_a[arm]),self.b_a[arm])
            std = np.sqrt(np.linalg.multi_dot([np.transpose(context[arm-1]),np.linalg.inv(self.A_a[arm]),context[arm-1]]))
            p_ta = np.dot(Theta_a.T,context[arm-1]) + self.alpha*std
            #print(p_ta)
            if not np.isnan(p_ta):
                arm_with_Q[arm] = p_ta
            
            
        highest = max(arm_with_Q.values())
        
        highest_Qs = [key for key, value in arm_with_Q.items() if value == highest]
        if len(highest_Qs)>1:
            action = np.random.choice(highest_Qs)#Tie Breaker
        else:
            #print(highest_Qs)
            action = highest_Qs[0]
        
        return action
        
            
            
        
        
        
    
    def update(self, arm, reward, context):
        #print("testing update")
        context = context.reshape(self.narms,self.ndims)
        
        
        if arm <= self.narms:
            context_Matrix = context[arm-1].reshape(-1,1)
            context_times_contextT = np.dot(context_Matrix,context_Matrix.T)
            
            self.A_a[arm] = np.add(self.A_a[arm],context_times_contextT)
            
            self.b_a[arm] = np.add(self.b_a[arm],np.dot(reward,context[arm-1]))
            
            
        
        
    

In [19]:
mab = LinUCB(10, 10, 1.0)
results_LinUCB = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('LinUCB average reward', np.mean(results_LinUCB))

LinUCB average reward 0.53


## 5. Contextual Bandits - LinThompson

In [ ]:
class LinThompson(MAB):
    """
    Contextual Thompson sampled multi-armed bandit (LinThompson)

    Arguments
    =========
    narms : int
        number of arms

    ndims : int
        number of dimensions for each arm's context

    v : float
        positive real explore-exploit parameter
    """
    
    def __init__(self, narms, ndims, v):
        
        
    def play(self, tround, context):
        
    
    def update(self, arm, reward, context):
        
    

In [ ]:
mab = LinThompson(10, 10, 1.0)
results_LinThompson = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('LinThompson average reward', np.mean(results_LinThompson))

## 6. Evaluation
### 6.A.

In [ ]:
EpsGreedy_reward = []
UCB_reward = []
betaThompson_reward = []
linUCB_reward = []
total_round = []
count = 0
for round in range(1,801):
    count +=1
    total_round.append(count)
    if round ==1:
        EpsGreedy_reward.append(results_EpsGreedy[round-1]/round)
        UCB_reward.append(results_UCB[round-1]/round)
        betaThompson_reward.append(results_BetaThompson[round-1]/round)
        linUCB_reward.append(results_LinUCB[round-1]/round)
    else:
        results_EpsGreedy[round-1] += results_EpsGreedy[round-2] 
        EpsGreedy_reward.append(results_EpsGreedy[round-1]/round)
        
        results_UCB[round-1]+= results_UCB[round-2]
        UCB_reward.append(results_UCB[round-1]/round)
        
        results_BetaThompson[round-1]+= results_BetaThompson[round-2]
        betaThompson_reward.append(results_BetaThompson[round-1]/round)
        
        results_LinUCB[round-1]+= results_LinUCB[round-2]
        linUCB_reward.append(results_LinUCB[round-1]/round)
        
plt.plot(total_round,EpsGreedy_reward, label = "eGreedy")
plt.plot(total_round,UCB_reward, label = "UCB")
plt.plot(total_round,betaThompson_reward, label = "BetaThompson")
plt.plot(total_round,linUCB_reward, label = "LinUCB")
plt.ylabel('Per-Round Cumulative Reward')
plt.xlabel('Rounds')
plt.legend()
plt.show()

### 6.B.

In [ ]:

.